In [1]:
#!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.20 KiB | 805.00 KiB/s, done.
Resolving deltas: 100% (185/185), done.


# Imports

In [4]:
import pandas as pd
import tarfile
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os

In [5]:
#os.environ['KAGGLE_USERNAME'] = 'axelmpm'
#os.environ['KAGGLE_KEY'] = '7ec546262604be71b856d29b2597ac36'
#!kaggle competitions download -c nlp-getting-started

In [34]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

# Formateo data con el formato que BERT espera

In [35]:
train = train.fillna(' ')
train['alpha'] = 'a'
train.index = train.id
train['text'] = train['keyword'] + ' ' + train['location'] + ' ' + train['text']
train['label'] = train['target']
train = train[['id','label','alpha','text']]

test = test.fillna(' ')
test.index = test.id
test['text'] = test['keyword'] + ' ' + test['location'] + ' ' + test['text']
test = test[['id','text']]

# Equilibro la data entre los labels para evitar biases

In [36]:
tag_1 = train[train['label'] == 1]
tag_0 = train[train['label'] == 0]

len(tag_1),len(tag_0)

(3271, 4342)

In [37]:
#Esto es para que el set de train este equilibrado entre 1 y 0 y para que el 
#set de validacion tenga solo datos nuevos que no se usaron en train 
#y que a la vez no se termine dejando ningun dato sin usar

tag_1_dev_size = 0.2

tag_0_dev_size = 1 - (len(tag_1) * (1 - tag_1_dev_size))/(len(tag_0))

tag_1_train, tag_1_dev = train_test_split(tag_1, test_size = tag_1_dev_size)
tag_0_train, tag_0_dev = train_test_split(tag_0, test_size = tag_0_dev_size)

train = pd.concat([tag_1_train,tag_0_train])
dev = pd.concat([tag_1_dev,tag_0_dev])
len(train),len(dev)

(5232, 2381)

# Guardar data en formato tsv para BERT

In [40]:
train.to_csv('data/bert_data/train.tsv', sep='\t', index=False, header=False)
dev.to_csv('data/bert_data/dev.tsv', sep='\t', index=False, header=False)
test.to_csv('data/bert_data/test.tsv', sep='\t', index=False, header=False)

In [44]:
!python bert/run_classifier.py 
--task_name=cola 
--do_train=true 
--do_eval=true 
--do_predict=true 
--data_dir=./data/ 
--vocab_file=./cased_L-12_H-768_A-12/vocab.txt 
--bert_config_file=./cased_L-12_H-768_A-12/bert_config.json 
--init_checkpoint=./cased_L-12_H-768_A-12/bert_model.ckpt 
--max_seq_length=400 
--train_batch_size=32 
--learning_rate=2e-5 
--num_train_epochs=1.0 
--output_dir=./bert_output/ 
--do_lower_case=False
--save_checkpoints_steps=9999999

SyntaxError: invalid syntax (<ipython-input-44-5c4bee79568f>, line 6)

# Referencia

    https://towardsml.com/2019/09/17/bert-explained-a-complete-guide-with-theory-and-tutorial/